# Desafio Técnico - Cientista de Dados Júnior
## Escritório de Dados - Prefeitura do Rio de Janeiro
#### Makalister Andrade da Silva


In [ ]:
!pip install basedosdados sqlalchemy pandas
import basedosdados as bd
import pandas as pd
from sqlalchemy import create_engine
from google.cloud import bigquery


### Todas as consultas foram realizadas no dia 11/02/2024, e portanto seus resultados devem levar em consideração o conteúdo do banco de dados nesta data

### Importando tabelas e salvando-as em dataframes

#### Importando a tabela de Chamados e salvando em dataframe para uso posterior

In [ ]:
query_chamados = "SELECT * FROM `datario.administracao_servicos_publicos.chamado_1746` WHERE data_inicio BETWEEN '2023-04-01T00:00:00' AND '2023-04-01T23:59:59'"
df_chamados = bd.read_sql(query_chamados, billing_project_id="emd-datario")

Downloading: 100%|██████████| 73/73 [00:00<00:00, 422.66rows/s]


#### Consultando e armazenando os dados de ``bairro`` em um dataframe:

In [ ]:
query_bairros = "SELECT * FROM `datario.dados_mestres.bairro`"
df_bairros = bd.read_sql(query_bairros, billing_project_id="emd-datario")

Downloading: 100%|██████████| 164/164 [00:03<00:00, 53.04rows/s]


#### Importando a tabela de Ocupação Hoteleira em Grandes Eventos no Rio

In [ ]:
query_eventos = "SELECT * FROM `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos`"
df_eventos = bd.read_sql(query_eventos, billing_project_id="emd-datario")

Downloading: 100%|██████████| 4/4 [00:00<00:00, 22.58rows/s]


### Questão 1

Quantos chamados foram abertos no dia 01/04/2023?

In [ ]:

query_conta_chamados = "SELECT count(id_chamado) AS QTD_Chamados FROM `datario.administracao_servicos_publicos.chamado_1746` WHERE data_inicio BETWEEN '2023-04-01T00:00:00' AND '2023-04-01T23:59:59'"
df_conta_chamados = bd.read_sql(query_conta_chamados, billing_project_id="emd-datario")

Downloading: 100%|██████████| 1/1 [00:00<00:00,  3.50rows/s]


In [ ]:
df_conta_chamados

,QTD_Chamados
0,73


``Resposta:`` Foram feitos 73 chamados no dia 01/04/2023.

### Questão 2

Qual o tipo de chamado que teve mais reclamações no dia 01/04/2023?

Listando as colunas do dataframe para identificação visual:

In [ ]:
sorted(df_chamados.keys())

['categoria',
 'data_alvo_diagnostico',
 'data_alvo_finalizacao',
 'data_fim',
 'data_inicio',
 'data_particao',
 'data_real_diagnostico',
 'dentro_prazo',
 'geometry',
 'id_bairro',
 'id_chamado',
 'id_logradouro',
 'id_subtipo',
 'id_territorialidade',
 'id_tipo',
 'id_unidade_organizacional',
 'id_unidade_organizacional_mae',
 'justificativa_status',
 'latitude',
 'longitude',
 'nome_unidade_organizacional',
 'numero_logradouro',
 'prazo_tipo',
 'prazo_unidade',
 'reclamacoes',
 'situacao',
 'status',
 'subtipo',
 'tempo_prazo',
 'tipo',
 'tipo_situacao',
 'unidade_organizacional_ouvidoria']

A coluna ``'tipo'`` conterá a informação para responder esta questão.

In [ ]:
df_chamados['tipo'].value_counts()

Poluição sonora             24
Estacionamento Irregular     9
Iluminação Pública           8
Remoção Gratuita             5
Limpeza                      5
Postura Municipal            4
Ocupação de área pública     3
Pavimentação                 3
Comlurb - Vetores            2
Fiscalização de obras        2
Manejo Arbóreo               2
Programa Cegonha Carioca     1
Ônibus                       1
Drenagem e Saneamento        1
Feiras                       1
Comércio ambulante           1
Semáforo                     1
Name: tipo, dtype: int64

``Resposta:`` Podemos verificar que o tipo de chamado mais frequente no dia 01/04/2023 é ``Poluição sonora``, contendo 24 chamados

### Questão 3

Quais os nomes dos 3 bairros que mais tiveram chamados abertos nesse dia?

Contando os bairros (``'id_bairro'``) que mais se repetem entre os chamados:

In [ ]:
id_bairros_max_chamados = df_chamados['id_bairro'].value_counts().nlargest(3).index.tolist()
id_bairros_max_chamados

['66', '26', '144']

In [ ]:
bairros_max_chamados = df_bairros.loc[df_bairros['id_bairro'].isin(id_bairros_max_chamados)]['nome'].tolist()
bairros_max_chamados

['Leblon', 'Engenho de Dentro', 'Campo Grande']

``Resposta:`` Os bairros que mais possuem chamados  são: ``'Leblon'``, ``'Engenho de Dentro'``, ``'Campo Grande'``, no dia 01/04/2023

### Questão 4

Qual o nome da subprefeitura com mais chamados abertos nesse dia?

In [ ]:
subprefeitura_max_chamados = df_bairros.loc[df_bairros['id_bairro'].isin([id_bairros_max_chamados[0]])]['subprefeitura'].tolist()
subprefeitura_max_chamados

['Zona Norte']

``Resposta:`` A subprefeitura com maior número de chamados no dia 01/04/2023 foi a da ``'Zona Norte'``

### Questão 5

Existe algum chamado aberto nesse dia que não foi associado a um bairro ou subprefeitura na tabela de bairros? Se sim, por que isso acontece?

In [ ]:
chamados_sem_b_sub = df_chamados[df_chamados['id_bairro'].isna()].to_dict()
{'tipo': chamados_sem_b_sub['tipo'][0],'subtipo': chamados_sem_b_sub['subtipo'][0]}

{'tipo': 'Ônibus',
 'subtipo': 'Verificação de ar condicionado inoperante no ônibus'}

``Resposta:`` O chamado não foi associado a um bairro ou subprefeitura pois o registro é sobre  ``'Verificação de ar condicionado inoperante no ônibus'``, logo não fora atribuído a nenhuma localidade.

### Questão 6

Quantos chamados com o subtipo "Perturbação do sossego" foram abertos desde 01/01/2022 até 31/12/2023 (incluindo extremidades)?

In [ ]:
total_chamados_perturbacao_sossego = df_chamados[(df_chamados['subtipo'] == 'Perturbação do sossego') &
                                                 (df_chamados['data_inicio'] >= '2022-01-01') &
                                                 (df_chamados['data_inicio'] <= '2023-12-31')].shape[0]

In [ ]:
total_chamados_perturbacao_sossego

24

``Resposta:`` Foram feitos 24 chamados relacionados a ``'Pertubação do sossego'``.

### Questão 7

Selecione os chamados com esse subtipo que foram abertos durante os eventos contidos na tabela de eventos (Reveillon, Carnaval e Rock in Rio).

In [ ]:

# Sua consulta SQL atualizada
query_tot_eventos = """
SELECT e.evento, COUNT(c.id_chamado) AS total_ocorrencias
FROM `datario.administracao_servicos_publicos.chamado_1746` c
INNER JOIN `datario.turismo_fluxo_visitantes.rede_hoteleira_ocupacao_eventos` e
ON c.data_inicio BETWEEN e.data_inicial AND e.data_final
WHERE c.subtipo = 'Perturbação do sossego'
AND e.evento IN ('Reveillon', 'Carnaval', 'Rock in Rio')
GROUP BY e.evento
ORDER BY total_ocorrencias DESC;
"""

client = bigquery.Client(project="emd-datario")

# Executa a consulta atualizada e converte os resultados para um DataFrame
df = client.query(query_tot_eventos).to_dataframe()

# Exibe o DataFrame
print(df)


        evento  total_ocorrencias
0  Rock in Rio                518
1     Carnaval                197
2    Reveillon                 79


``Resposta:``

1.   Rock in Rio 518
2.   Carnaval 197
3.   Reveillon 79.